In [4]:
print("awan")

awan


In [5]:
pwd

'e:\\chatbot\\Chatbot\\research'

In [9]:
import os
os.chdir("../")

In [10]:
pwd

'e:\\chatbot\\Chatbot'

In [13]:
from langchain.document_loaders import PyPDFLoader

ModuleNotFoundError: No module named 'langchain.document_loaders'

In [17]:
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [19]:
def load_pdf(data):
    loader = DirectoryLoader(
        data,
        glob="*.pdf",
        loader_cls= PyPDFLoader
    )

    document = loader.load()

    return document

In [20]:
extracted_doc = load_pdf("data")

In [24]:
len(extracted_doc)

93

In [32]:
from typing import List
from langchain_core.documents import Document

In [37]:
extracted_doc

[Document(metadata={'producer': 'Adobe PDF Library 17.0', 'creator': 'Adobe InDesign 19.1 (Windows)', 'creationdate': '2025-02-14T12:31:53+05:00', 'moddate': '2025-02-14T12:33:59+05:00', 'trapped': '/False', 'source': 'data\\agent_ai.pdf', 'total_pages': 93, 'page': 0, 'page_label': '1'}, page_content=''),
 Document(metadata={'producer': 'Adobe PDF Library 17.0', 'creator': 'Adobe InDesign 19.1 (Windows)', 'creationdate': '2025-02-14T12:31:53+05:00', 'moddate': '2025-02-14T12:33:59+05:00', 'trapped': '/False', 'source': 'data\\agent_ai.pdf', 'total_pages': 93, 'page': 1, 'page_label': '2'}, page_content='2\nMastering AI Agents\nPreface\nIn our previous e-book, “Mastering RAG,” our goal was clear: building enterprise-grade \nRAG systems, productionizing them, monitoring their performance, and improving them. \nAt the core of it, we understood how RAG systems enhance an LLM’s ability to work with \nspecific knowledge by providing relevant context. \nIn this e-book, we’re taking a step fu

In [38]:
def extract_meta_data(docs: List[Document]) -> List[Document]:

    min_docs: List[Document] = []

    for doc in docs:
        src = doc.metadata.get("source" , "page")
        min_docs.append(
            Document(
                page_content=doc.page_content,
                metadata = {"source" : src}

            )
        )
    return min_docs

In [39]:
minimal_doc = extract_meta_data(extracted_doc)

In [41]:
minimal_doc[:5]

[Document(metadata={'source': 'data\\agent_ai.pdf'}, page_content=''),
 Document(metadata={'source': 'data\\agent_ai.pdf'}, page_content='2\nMastering AI Agents\nPreface\nIn our previous e-book, “Mastering RAG,” our goal was clear: building enterprise-grade \nRAG systems, productionizing them, monitoring their performance, and improving them. \nAt the core of it, we understood how RAG systems enhance an LLM’s ability to work with \nspecific knowledge by providing relevant context. \nIn this e-book, we’re taking a step further and asking, “How do we use LLMs to \naccomplish end-to-end tasks?” This singular question opens up a door: AI agents. A RAG \nsystem helps an LLM provide accurate answers based on given context. An AI agent \ntakes that answer and actually does something with it — makes decisions, executes \ntasks, or coordinates multiple steps to achieve a goal.\nA RAG-enhanced LLM could help answer questions about policy details by pulling relevant \ninformation. But an AI agent

In [42]:
#Lets do a chunking


In [49]:
def chunking(minimal_doc):
    text_chunk = RecursiveCharacterTextSplitter(
        chunk_size=100,
        chunk_overlap=20
    )

    split_chunks = text_chunk.split_documents(minimal_doc)

    return split_chunks



In [50]:
chunked_doc = chunking(minimal_doc)

In [51]:
len(chunked_doc)

1507